In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

# Importing the dataset

In [2]:
import pandas as pd
df = pd.read_excel (r'data.xlsx')
print (df)

               Date first seen    Duration  Proto    Src IP Addr  Src Pt  \
0      2017-03-22 00:00:14.461       0.000  TCP         19394_40   54719   
1      2017-03-22 00:00:14.461       0.000  TCP       EXT_SERVER      23   
2      2017-03-22 00:00:33.525       0.000  TCP         19395_18    4622   
3      2017-03-22 00:00:33.525       0.000  TCP       EXT_SERVER    7547   
4      2017-03-22 00:00:40.205       0.000  TCP        11246_253    6000   
...                        ...         ...    ...            ...     ...   
159368 2017-03-22 17:07:28.444  576387.261  TCP    OPENSTACK_NET   54997   
159369 2017-03-22 16:52:38.738  577308.174  TCP       EXT_SERVER    8082   
159370 2017-03-22 16:52:38.738  577308.174  TCP    OPENSTACK_NET   54695   
159371 2017-03-22 17:00:15.265  576881.851  TCP       EXT_SERVER    8082   
159372 2017-03-22 17:00:15.265  576881.851  TCP    OPENSTACK_NET   54853   

          Dst IP Addr   Dst Pt  Packets     Bytes  Flows   Flags  Tos  \
0          EXT

# Displaying the data

In [3]:
df.head(80)

,Date first seen,Duration,Proto,Src IP Addr,Src Pt,Dst IP Addr,Dst Pt,Packets,Bytes,Flows,Flags,Tos,class,attackType,attackID,attackDescription
0,2017-03-22 00:00:14.461,0.000,TCP,19394_40,54719,EXT_SERVER,23.0,1,46,1,....S.,0,suspicious,---,---,---
1,2017-03-22 00:00:14.461,0.000,TCP,EXT_SERVER,23,19394_40,54719.0,1,40,1,.A.R..,0,suspicious,---,---,---
2,2017-03-22 00:00:33.525,0.000,TCP,19395_18,4622,EXT_SERVER,7547.0,1,46,1,....S.,0,suspicious,---,---,---
3,2017-03-22 00:00:33.525,0.000,TCP,EXT_SERVER,7547,19395_18,4622.0,1,40,1,.A.R..,0,suspicious,---,---,---
4,2017-03-22 00:00:40.205,0.000,TCP,11246_253,6000,EXT_SERVER,3306.0,1,46,1,....S.,0,suspicious,---,---,---
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,2017-03-22 00:04:40.058,0.057,TCP,OPENSTACK_NET,60805,EXT_SERVER,8000.0,6,515,1,.AP.SF,0,normal,---,---,---
76,2017-03-22 00:04:40.058,0.068,TCP,EXT_SERVER,8000,OPENSTACK_NET,60806.0,7,702,1,.AP.SF,0,normal,---,---,---
77,2017-03-22 00:04:40.058,0.068,TCP,OPENSTACK_NET,60806,EXT_SERVER,8000.0,6,586,1,.AP.SF,0,normal,---,---,---
78,2017-03-22 00:04:40.558,0.056,TCP,EXT_SERVER,8000,OPENSTACK_NET,51179.0,7,702,1,.AP.SF,0,normal,---,---,---


# Displaying count in each type of attack

In [4]:
print(df['class'].value_counts())

suspicious    116588
normal         28436
unknown         9571
attacker        2389
victim          2389
Name: class, dtype: int64


# Preprocessing

In [5]:
df.head(20)

,Date first seen,Duration,Proto,Src IP Addr,Src Pt,Dst IP Addr,Dst Pt,Packets,Bytes,Flows,Flags,Tos,class,attackType,attackID,attackDescription
0,2017-03-22 00:00:14.461,0.000,TCP,19394_40,54719,EXT_SERVER,23.0,1,46,1,....S.,0,suspicious,---,---,---
1,2017-03-22 00:00:14.461,0.000,TCP,EXT_SERVER,23,19394_40,54719.0,1,40,1,.A.R..,0,suspicious,---,---,---
2,2017-03-22 00:00:33.525,0.000,TCP,19395_18,4622,EXT_SERVER,7547.0,1,46,1,....S.,0,suspicious,---,---,---
3,2017-03-22 00:00:33.525,0.000,TCP,EXT_SERVER,7547,19395_18,4622.0,1,40,1,.A.R..,0,suspicious,---,---,---
4,2017-03-22 00:00:40.205,0.000,TCP,11246_253,6000,EXT_SERVER,3306.0,1,46,1,....S.,0,suspicious,---,---,---
5,2017-03-22 00:00:40.205,0.000,TCP,EXT_SERVER,3306,11246_253,6000.0,1,40,1,.A.R..,0,suspicious,---,---,---
6,2017-03-22 00:00:38.395,5.514,TCP,EXT_SERVER,22,19377_254,7432.0,10,2445,1,.AP.SF,0,suspicious,---,---,---
7,2017-03-22 00:00:38.395,5.514,TCP,19377_254,7432,EXT_SERVER,22.0,12,1631,1,.APRSF,0,suspicious,---,---,---
8,2017-03-22 00:00:20.548,37.611,TCP,18851_139,35854,EXT_SERVER,22.0,14,2073,1,.APRS.,0,suspicious,---,---,---
9,2017-03-22 00:00:20.548,37.611,TCP,EXT_SERVER,22,18851_139,35854.0,26,4949,1,.AP.S.,0,suspicious,---,---,---


attackType,attackID,attackDescription have no values in them so we drop those columns

In [6]:
df=df.drop(['attackType','attackID','attackDescription'], axis = 1)

We need to see which columns have same values across all the rows as they do not provide for the ML model


In [7]:
for (columnName, columnData) in df.iteritems():
    print("   ")
    print('Column Name : ',columnName)
    print('Column Range: ',df[columnName].value_counts())

   
Column Name :  Date first seen
Column Range:  2017-03-23 09:47:20.284    100
2017-03-23 09:47:20.256    100
2017-03-23 09:47:20.279    100
2017-03-23 09:47:20.258    100
2017-03-23 09:47:20.293    100
                          ... 
2017-03-23 09:18:53.209      1
2017-03-23 18:25:12.154      1
2017-03-22 02:35:07.863      1
2017-03-28 02:03:26.553      1
2017-03-26 07:37:37.303      1
Name: Date first seen, Length: 76360, dtype: int64
   
Column Name :  Duration
Column Range:  0.000      23269
30.999      1745
30.998      1704
30.997      1684
0.001       1636
           ...  
74.993         1
3.089          1
591.612        1
4.218          1
3.026          1
Name: Duration, Length: 15230, dtype: int64
   
Column Name :  Proto
Column Range:  TCP      154591
UDP        2869
ICMP       1878
GRE          35
Name: Proto, dtype: int64
   
Column Name :  Src IP Addr
Column Range:  EXT_SERVER       79288
OPENSTACK_NET    14218
21428_11         11840
10006_27          5469
10979_30        

Dropping Tos and Flows have only one value

In [8]:
df=df.drop(['Tos','Flows'], axis = 1)

In [9]:
df

,Date first seen,Duration,Proto,Src IP Addr,Src Pt,Dst IP Addr,Dst Pt,Packets,Bytes,Flags,class
0,2017-03-22 00:00:14.461,0.000,TCP,19394_40,54719,EXT_SERVER,23.0,1,46,....S.,suspicious
1,2017-03-22 00:00:14.461,0.000,TCP,EXT_SERVER,23,19394_40,54719.0,1,40,.A.R..,suspicious
2,2017-03-22 00:00:33.525,0.000,TCP,19395_18,4622,EXT_SERVER,7547.0,1,46,....S.,suspicious
3,2017-03-22 00:00:33.525,0.000,TCP,EXT_SERVER,7547,19395_18,4622.0,1,40,.A.R..,suspicious
4,2017-03-22 00:00:40.205,0.000,TCP,11246_253,6000,EXT_SERVER,3306.0,1,46,....S.,suspicious
...,...,...,...,...,...,...,...,...,...,...,...
159368,2017-03-22 17:07:28.444,576387.261,TCP,OPENSTACK_NET,54997,EXT_SERVER,8082.0,15010,7.2 M,.AP.S.,normal
159369,2017-03-22 16:52:38.738,577308.174,TCP,EXT_SERVER,8082,OPENSTACK_NET,54695.0,11650,5.5 M,.AP.S.,normal
159370,2017-03-22 16:52:38.738,577308.174,TCP,OPENSTACK_NET,54695,EXT_SERVER,8082.0,18571,13.1 M,.AP.S.,normal
159371,2017-03-22 17:00:15.265,576881.851,TCP,EXT_SERVER,8082,OPENSTACK_NET,54853.0,13176,13.0 M,.AP.S.,normal


In [10]:
for (columnName, columnData) in df.iteritems():
    print("   ")
    print('Column Name : ',columnName)
    print('Column Range: ',df[columnName].value_counts())

   
Column Name :  Date first seen
Column Range:  2017-03-23 09:47:20.284    100
2017-03-23 09:47:20.256    100
2017-03-23 09:47:20.279    100
2017-03-23 09:47:20.258    100
2017-03-23 09:47:20.293    100
                          ... 
2017-03-23 09:18:53.209      1
2017-03-23 18:25:12.154      1
2017-03-22 02:35:07.863      1
2017-03-28 02:03:26.553      1
2017-03-26 07:37:37.303      1
Name: Date first seen, Length: 76360, dtype: int64
   
Column Name :  Duration
Column Range:  0.000      23269
30.999      1745
30.998      1704
30.997      1684
0.001       1636
           ...  
74.993         1
3.089          1
591.612        1
4.218          1
3.026          1
Name: Duration, Length: 15230, dtype: int64
   
Column Name :  Proto
Column Range:  TCP      154591
UDP        2869
ICMP       1878
GRE          35
Name: Proto, dtype: int64
   
Column Name :  Src IP Addr
Column Range:  EXT_SERVER       79288
OPENSTACK_NET    14218
21428_11         11840
10006_27          5469
10979_30        

In [11]:
for (columnName, columnData) in df.iteritems():
    print("   ")
    print('Column Name : ',columnName)
    print(df[columnName].dtypes)


   
Column Name :  Date first seen
datetime64[ns]
   
Column Name :  Duration
float64
   
Column Name :  Proto
object
   
Column Name :  Src IP Addr
object
   
Column Name :  Src Pt
int64
   
Column Name :  Dst IP Addr
object
   
Column Name :  Dst Pt
float64
   
Column Name :  Packets
int64
   
Column Name :  Bytes
object
   
Column Name :  Flags
object
   
Column Name :  class
object


In [12]:
print(df.describe())

            Duration         Src Pt         Dst Pt        Packets
count  159373.000000  159373.000000  159373.000000  159373.000000
mean       79.663156   21719.480031   21552.513306      18.475074
std      5552.439126   23472.037681   23481.807040     701.826927
min         0.000000       0.000000       0.000000       1.000000
25%         0.064000      22.000000      22.000000       6.000000
50%         9.072000    8000.000000    8000.000000      15.000000
75%        10.661000   45759.000000   45588.000000      19.000000
max    584199.578000   65535.000000   65535.000000  119128.000000


In [13]:
print(df.isna().sum())
print("\n")

Date first seen    0
Duration           0
Proto              0
Src IP Addr        0
Src Pt             0
Dst IP Addr        0
Dst Pt             0
Packets            0
Bytes              0
Flags              0
class              0
dtype: int64




In [14]:
from sklearn.preprocessing import LabelEncoder
labelEncoder = LabelEncoder()

In [15]:
labelEncoder.fit(df['Proto'])

LabelEncoder()

In [16]:
df['Proto'] = labelEncoder.transform(df['Proto'])

In [17]:
df

,Date first seen,Duration,Proto,Src IP Addr,Src Pt,Dst IP Addr,Dst Pt,Packets,Bytes,Flags,class
0,2017-03-22 00:00:14.461,0.000,2,19394_40,54719,EXT_SERVER,23.0,1,46,....S.,suspicious
1,2017-03-22 00:00:14.461,0.000,2,EXT_SERVER,23,19394_40,54719.0,1,40,.A.R..,suspicious
2,2017-03-22 00:00:33.525,0.000,2,19395_18,4622,EXT_SERVER,7547.0,1,46,....S.,suspicious
3,2017-03-22 00:00:33.525,0.000,2,EXT_SERVER,7547,19395_18,4622.0,1,40,.A.R..,suspicious
4,2017-03-22 00:00:40.205,0.000,2,11246_253,6000,EXT_SERVER,3306.0,1,46,....S.,suspicious
...,...,...,...,...,...,...,...,...,...,...,...
159368,2017-03-22 17:07:28.444,576387.261,2,OPENSTACK_NET,54997,EXT_SERVER,8082.0,15010,7.2 M,.AP.S.,normal
159369,2017-03-22 16:52:38.738,577308.174,2,EXT_SERVER,8082,OPENSTACK_NET,54695.0,11650,5.5 M,.AP.S.,normal
159370,2017-03-22 16:52:38.738,577308.174,2,OPENSTACK_NET,54695,EXT_SERVER,8082.0,18571,13.1 M,.AP.S.,normal
159371,2017-03-22 17:00:15.265,576881.851,2,EXT_SERVER,8082,OPENSTACK_NET,54853.0,13176,13.0 M,.AP.S.,normal


In [18]:
labelEncoder2=LabelEncoder()
labelEncoder2.fit(df['Src IP Addr'])
df['Src IP Addr']=labelEncoder2.transform(df['Src IP Addr'])

In [19]:
df

,Date first seen,Duration,Proto,Src IP Addr,Src Pt,Dst IP Addr,Dst Pt,Packets,Bytes,Flags,class
0,2017-03-22 00:00:14.461,0.000,2,1663,54719,EXT_SERVER,23.0,1,46,....S.,suspicious
1,2017-03-22 00:00:14.461,0.000,2,9482,23,19394_40,54719.0,1,40,.A.R..,suspicious
2,2017-03-22 00:00:33.525,0.000,2,1664,4622,EXT_SERVER,7547.0,1,46,....S.,suspicious
3,2017-03-22 00:00:33.525,0.000,2,9482,7547,19395_18,4622.0,1,40,.A.R..,suspicious
4,2017-03-22 00:00:40.205,0.000,2,646,6000,EXT_SERVER,3306.0,1,46,....S.,suspicious
...,...,...,...,...,...,...,...,...,...,...,...
159368,2017-03-22 17:07:28.444,576387.261,2,9483,54997,EXT_SERVER,8082.0,15010,7.2 M,.AP.S.,normal
159369,2017-03-22 16:52:38.738,577308.174,2,9482,8082,OPENSTACK_NET,54695.0,11650,5.5 M,.AP.S.,normal
159370,2017-03-22 16:52:38.738,577308.174,2,9483,54695,EXT_SERVER,8082.0,18571,13.1 M,.AP.S.,normal
159371,2017-03-22 17:00:15.265,576881.851,2,9482,8082,OPENSTACK_NET,54853.0,13176,13.0 M,.AP.S.,normal


In [20]:
labelEncoder3=LabelEncoder()
labelEncoder3.fit(df['Dst IP Addr'])
df['Dst IP Addr']=labelEncoder3.transform(df['Dst IP Addr'])

In [21]:
df

,Date first seen,Duration,Proto,Src IP Addr,Src Pt,Dst IP Addr,Dst Pt,Packets,Bytes,Flags,class
0,2017-03-22 00:00:14.461,0.000,2,1663,54719,9440,23.0,1,46,....S.,suspicious
1,2017-03-22 00:00:14.461,0.000,2,9482,23,1657,54719.0,1,40,.A.R..,suspicious
2,2017-03-22 00:00:33.525,0.000,2,1664,4622,9440,7547.0,1,46,....S.,suspicious
3,2017-03-22 00:00:33.525,0.000,2,9482,7547,1658,4622.0,1,40,.A.R..,suspicious
4,2017-03-22 00:00:40.205,0.000,2,646,6000,9440,3306.0,1,46,....S.,suspicious
...,...,...,...,...,...,...,...,...,...,...,...
159368,2017-03-22 17:07:28.444,576387.261,2,9483,54997,9440,8082.0,15010,7.2 M,.AP.S.,normal
159369,2017-03-22 16:52:38.738,577308.174,2,9482,8082,9441,54695.0,11650,5.5 M,.AP.S.,normal
159370,2017-03-22 16:52:38.738,577308.174,2,9483,54695,9440,8082.0,18571,13.1 M,.AP.S.,normal
159371,2017-03-22 17:00:15.265,576881.851,2,9482,8082,9441,54853.0,13176,13.0 M,.AP.S.,normal


In [22]:
df.dtypes

Date first seen    datetime64[ns]
Duration                  float64
Proto                       int32
Src IP Addr                 int32
Src Pt                      int64
Dst IP Addr                 int32
Dst Pt                    float64
Packets                     int64
Bytes                      object
Flags                      object
class                      object
dtype: object

In [23]:
labelEncoder4=LabelEncoder()
labelEncoder4.fit(df['Flags'])
df['Flags']=labelEncoder4.transform(df['Flags'])

In [24]:
df

,Date first seen,Duration,Proto,Src IP Addr,Src Pt,Dst IP Addr,Dst Pt,Packets,Bytes,Flags,class
0,2017-03-22 00:00:14.461,0.000,2,1663,54719,9440,23.0,1,46,11,suspicious
1,2017-03-22 00:00:14.461,0.000,2,9482,23,1657,54719.0,1,40,17,suspicious
2,2017-03-22 00:00:33.525,0.000,2,1664,4622,9440,7547.0,1,46,11,suspicious
3,2017-03-22 00:00:33.525,0.000,2,9482,7547,1658,4622.0,1,40,17,suspicious
4,2017-03-22 00:00:40.205,0.000,2,646,6000,9440,3306.0,1,46,11,suspicious
...,...,...,...,...,...,...,...,...,...,...,...
159368,2017-03-22 17:07:28.444,576387.261,2,9483,54997,9440,8082.0,15010,7.2 M,21,normal
159369,2017-03-22 16:52:38.738,577308.174,2,9482,8082,9441,54695.0,11650,5.5 M,21,normal
159370,2017-03-22 16:52:38.738,577308.174,2,9483,54695,9440,8082.0,18571,13.1 M,21,normal
159371,2017-03-22 17:00:15.265,576881.851,2,9482,8082,9441,54853.0,13176,13.0 M,21,normal


In [25]:
labelEncoder5=LabelEncoder()
labelEncoder5.fit(df['class'])
df['class']=labelEncoder5.transform(df['class'])

In [26]:
df

,Date first seen,Duration,Proto,Src IP Addr,Src Pt,Dst IP Addr,Dst Pt,Packets,Bytes,Flags,class
0,2017-03-22 00:00:14.461,0.000,2,1663,54719,9440,23.0,1,46,11,2
1,2017-03-22 00:00:14.461,0.000,2,9482,23,1657,54719.0,1,40,17,2
2,2017-03-22 00:00:33.525,0.000,2,1664,4622,9440,7547.0,1,46,11,2
3,2017-03-22 00:00:33.525,0.000,2,9482,7547,1658,4622.0,1,40,17,2
4,2017-03-22 00:00:40.205,0.000,2,646,6000,9440,3306.0,1,46,11,2
...,...,...,...,...,...,...,...,...,...,...,...
159368,2017-03-22 17:07:28.444,576387.261,2,9483,54997,9440,8082.0,15010,7.2 M,21,1
159369,2017-03-22 16:52:38.738,577308.174,2,9482,8082,9441,54695.0,11650,5.5 M,21,1
159370,2017-03-22 16:52:38.738,577308.174,2,9483,54695,9440,8082.0,18571,13.1 M,21,1
159371,2017-03-22 17:00:15.265,576881.851,2,9482,8082,9441,54853.0,13176,13.0 M,21,1


In [27]:
df.dtypes

Date first seen    datetime64[ns]
Duration                  float64
Proto                       int32
Src IP Addr                 int32
Src Pt                      int64
Dst IP Addr                 int32
Dst Pt                    float64
Packets                     int64
Bytes                      object
Flags                       int32
class                       int32
dtype: object

In [28]:
df['Bytes'] = df['Bytes'].astype(str)
labelEncoder6=LabelEncoder()
labelEncoder6.fit(df['Bytes'])
df['Bytes']=labelEncoder6.transform(df['Bytes'])

In [29]:
df.dtypes

Date first seen    datetime64[ns]
Duration                  float64
Proto                       int32
Src IP Addr                 int32
Src Pt                      int64
Dst IP Addr                 int32
Dst Pt                    float64
Packets                     int64
Bytes                       int32
Flags                       int32
class                       int32
dtype: object

In [30]:

labelEncoder7=LabelEncoder()
labelEncoder7.fit(df['Date first seen'])
df['Date first seen']=labelEncoder7.transform(df['Date first seen'])

In [31]:
df.dtypes

Date first seen      int64
Duration           float64
Proto                int32
Src IP Addr          int32
Src Pt               int64
Dst IP Addr          int32
Dst Pt             float64
Packets              int64
Bytes                int32
Flags                int32
class                int32
dtype: object

In [32]:
df

,Date first seen,Duration,Proto,Src IP Addr,Src Pt,Dst IP Addr,Dst Pt,Packets,Bytes,Flags,class
0,0,0.000,2,1663,54719,9440,23.0,1,2318,11,2
1,0,0.000,2,9482,23,1657,54719.0,1,1973,17,2
2,3,0.000,2,1664,4622,9440,7547.0,1,2318,11,2
3,3,0.000,2,9482,7547,1658,4622.0,1,1973,17,2
4,5,0.000,2,646,6000,9440,3306.0,1,2318,11,2
...,...,...,...,...,...,...,...,...,...,...,...
159368,6993,576387.261,2,9483,54997,9440,8082.0,15010,16,21,1
159369,6928,577308.174,2,9482,8082,9441,54695.0,11650,11,21,1
159370,6928,577308.174,2,9483,54695,9440,8082.0,18571,22,21,1
159371,6957,576881.851,2,9482,8082,9441,54853.0,13176,21,21,1


In [33]:
df = df.sample(frac = 1)

In [34]:
df

,Date first seen,Duration,Proto,Src IP Addr,Src Pt,Dst IP Addr,Dst Pt,Packets,Bytes,Flags,class
84942,40255,10.373,2,9482,22,33,34517.0,18,1341,22,2
55247,25863,8.900,2,3842,46777,9440,22.0,15,630,22,2
152774,73204,24.901,2,9482,22,600,10783.0,26,1810,24,2
9065,4221,8.939,2,4,43435,9440,22.0,15,630,22,2
10144,4826,0.309,2,9482,23,2100,54579.0,4,286,17,2
...,...,...,...,...,...,...,...,...,...,...,...
58807,27601,9.938,2,9482,22,3825,35310.0,19,1389,22,2
144358,69122,25.549,2,9482,22,600,47137.0,26,1810,24,2
104146,49577,0.000,2,7060,33829,9440,5358.0,1,2318,11,2
60639,28515,0.000,2,9482,23,4778,63793.0,1,1973,17,2


In [35]:
X = np.array(df.drop(['class'], 1).astype(float))

In [36]:
y = np.array(df['class'])

In [37]:
kmeans = KMeans(n_clusters=3) 
kmeans.fit(X)

KMeans(n_clusters=3)

In [38]:
correct = 0
labels=[]
for i in range(len(X)):
    predict_me = np.array(X[i].astype(float))
    predict_me = predict_me.reshape(-1, len(predict_me))
    prediction = kmeans.predict(predict_me)
    labels.append(prediction[0])
    if prediction[0] == y[i]:
        correct += 1

print(correct/len(X))

0.34982713508561675


# Homogeneity score

In [39]:
from sklearn.metrics import homogeneity_score
homogeneity_score(y, labels)

0.09745057262111752

# Completeness Score

In [40]:
from sklearn.metrics import completeness_score
print(completeness_score(y, labels))

0.07530585453116438


# V measure

In [45]:
from sklearn.metrics import v_measure_score
v_measure_score(y, labels)

0.08495890736748908